In [1]:
!pip install pytesseract

In [2]:
!pip install opencv-python

In [3]:
!pip install pymupdf

In [2]:
import os
import cv2
import fitz
import itertools
import pytesseract
import numpy as np
import pandas as pd
from PIL import Image

In [54]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [55]:
os.environ['PATH'] += r';C:\Program Files\Tesseract-OCR'

Обобщим обработку на случай pdf файла на входе, сделаем пердобработку и выполним нарезку

In [ ]:
class TableExtraction:
    self.input_data = []
    self.list_of_np_arrays = []
    self.threshold_images = []
    self.dilated_images = []

    def convert_file_to_array(self, file_path):
        file_extension = os.path.splitext(file_path)[1]
        if file_extension == '.pdf':
            doc = fitz.open(file_path)
            for n in range(doc.page_count):
                page = doc.load_page(n)
                pix = page.get_pixmap()
                image = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
                image = np.ascontiguousarray(image[..., [2, 1, 0]])
                self.list_of_np_arrays.append(image)
            else:
                raise ValueError('Unsupported file format')



In [ ]:
def convert_file_to_array(self, file_path):
    file_extension = os.path.splitext(file_path)[1]
    if file_extension == '.pdf':
        doc = fitz.open(file_path)
        for n in range(doc.page_count):
            page = doc.load_page(n)
            pix = page.get_pixmap()
            image = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
            image = np.ascontiguousarray(image[..., [2, 1, 0]])
            self.list_of_np_arrays.append(image)
    else:
        raise ValueError('Unsupported file format')

def convert_pdf_to_array(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfFileReader(file)
        num_pages = pdf_reader.numPages

        page_data = []
        for page_num in range(num_pages):
            page = pdf_reader.getPage(page_num)
            page_text = page.extractText().replace('\n', '')
            page_data.append(page_text)

        return np.array(page_data)

def convert_rtf_to_array(file_path):
    doc = docx.Document(file_path)
    paragraphs = [p.text for p in doc.paragraphs]
    return np.array(paragraphs)

def convert_djvu_to_array(file_path):
    # Добавьте здесь код для конвертации DJVU в текст или изображение,
    # а затем соответствующим образом преобразуйте в numpy array
    return np.array([])

# Пример использования
file_path = 'example.pdf'
file_array = convert_file_to_array(file_path)

print(file_array)

In [27]:
# In case of .pdf file
threshold_images = []
dilated_images = []
file = 'test.pdf'
doc = fitz.open(file)
for n in range(doc.page_count):
    page = doc.load_page(n)
    pix = page.get_pixmap()
    image = np.frombuffer(pix.samples,
                          dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    image = np.ascontiguousarray(image[..., [2, 1, 0]])
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Thresholding image, ret = value, thresh_value = new binary image
    _, threshold_image = cv2.threshold(gray_image, 180, 255, cv2.THRESH_BINARY_INV)
    threshold_images.append(threshold_image)
    # Dilating image some kind of binary blur
    kernel = np.ones((5, 5), np.uint8)
    dilated_image = cv2.dilate(threshold_image, kernel, iterations=1)
    dilated_images.append(dilated_image)

In [28]:
# Looking for contours, get list of contours and its hierarchy structure
contours, hierarchy = cv2.findContours(dilated_value, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [29]:
# Extracting coordinates and filtering them empirically
coordinates = []  # our target list of contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)  # return bounding box of contour
    if (h > 20 or w > 20) and (w / h > 2 or h / w > 2):
        coordinates.append((x, y, w, h))

coordinates

[(87, 82, 513, 176),
 (89, 225, 283, 31),
 (288, 232, 39, 10),
 (226, 232, 60, 10),
 (204, 232, 21, 8),
 (133, 232, 70, 10),
 (94, 229, 37, 13),
 (518, 209, 57, 15),
 (470, 209, 46, 15),
 (421, 209, 47, 15),
 (373, 209, 47, 15),
 (89, 209, 283, 15),
 (94, 212, 86, 10),
 (518, 193, 57, 15),
 (470, 193, 46, 15),
 (421, 193, 47, 15),
 (373, 193, 47, 15),
 (89, 193, 283, 15),
 (518, 176, 57, 16),
 (470, 176, 46, 16),
 (421, 176, 47, 16),
 (373, 176, 47, 16),
 (89, 176, 283, 16),
 (94, 180, 57, 10),
 (518, 160, 57, 15),
 (470, 160, 46, 15),
 (421, 160, 47, 15),
 (373, 160, 47, 15),
 (89, 160, 283, 15),
 (518, 144, 57, 15),
 (470, 144, 46, 15),
 (421, 144, 47, 15),
 (373, 144, 47, 15),
 (89, 144, 283, 15),
 (518, 128, 57, 15),
 (470, 128, 46, 15),
 (421, 128, 47, 15),
 (373, 128, 47, 15),
 (89, 128, 283, 15),
 (110, 120, 33, 7),
 (518, 111, 57, 16),
 (535, 114, 25, 11),
 (470, 111, 46, 16),
 (481, 114, 25, 11),
 (421, 111, 47, 16),
 (433, 114, 26, 11),
 (373, 111, 47, 16),
 (385, 114, 26, 11

In [25]:
# Загрузка изображения
image = cv2.imread('image.png')

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Адаптивная бинаризация для выделения текста
binary_image = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

# Фильтрация контуров для удаления шумов
contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
filtered_contours = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w > 10 and h > 10:  # Фильтрация маленьких контуров
        filtered_contours.append(contour)

print(filtered_contours)

# Создание маски для удаления текста
mask = np.ones(gray_image.shape, dtype=np.uint8) * 255
cv2.drawContours(mask, filtered_contours, -1, 0, thickness=cv2.FILLED)

# Применение маски к исходному изображению
result = cv2.bitwise_and(image, image, mask=mask)

# Отображение результата
cv2.imshow('Результат', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

[array([[[  8,   6]],

       [[  8, 176]],

       [[839, 176]],

       [[839,   6]]], dtype=int32)]


In [26]:
# Загрузка изображения
image = cv2.imread('image.png')

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Получение ограничивающих рамок символов
custom_config = r'--oem 1 --psm 1 -l rus'
boxes = pytesseract.image_to_boxes(gray_image, config=custom_config)

# Разделение строки на список строк
box_lines = boxes.strip().split('\n')

# Преобразование каждой строки в список
boxes_list = [line.split() for line in box_lines]

# Так как cv2 и tesseract используют разные системы координат
height, width, _ = image.shape

# Список границ таблицы
borders = []

# Вывод ограничивающих рамок символов
for box in boxes_list:
    if box[0] == '~':
        x, y, w, h = int(box[1]), int(box[2]), int(box[3]), int(box[4])
        borders.append([x, y, w, h])
        cv2.rectangle(image, (x, height - y), (w, height - h), (0, 255, 255), 1)

# # Отображение изображения с ограничивающими рамками символов
# cv2.imshow('Image with Character Boxes', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [323]:
# Находим все вертикальные и горизонтальные линии каркаса таблицы
vertical_lines = []
horizontal_lines = []

# Параметры допуска при котором считаем что линии ровные и пересекаются
epsilon = 3

for x1, y1, x2, y2 in borders:
    if abs(x1 - x2) < epsilon:  # Вертикальная линия
        vertical_lines.append((x1, y1, x2, y2))
    elif abs(y1 - y2) < epsilon:  # Горизонтальная линия
        horizontal_lines.append((x1, y1, x2, y2))

# Находим координаты узлов таблицы
table_nodes = []
for v_line, h_line in itertools.product(vertical_lines, horizontal_lines):
    x, y = v_line[0], h_line[1]
    # cv2.rectangle(image, (x, height - y), (x + 3, height - y + 3), (0, 0, 255), 1)
    table_nodes.append((x, y))

# # Отображение изображения
# cv2.imshow('Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Координаты узлов таблицы:
(8, 178)
(8, 65)
(8, 37)
(8, 8)
(177, 178)
(177, 65)
(177, 37)
(177, 8)
(424, 178)
(424, 65)
(424, 37)
(424, 8)
(670, 178)
(670, 65)
(670, 37)
(670, 8)
(839, 178)
(839, 65)
(839, 37)
(839, 8)


In [324]:
# Нахождение количества уникальных значений координат x и y
unique_x = set(x for x, y in table_nodes)
unique_y = set(y for x, y in table_nodes)

# Определение числа строк и столбцов таблицы
num_rows = len(unique_y) - 1
num_columns = len(unique_x) - 1

# Отсортируем узлы по оси x
nodes_sorted_x = sorted(table_nodes, key=lambda x: (x[0], x[1]))

# Создадим список прямоугольников
rectangles = []

# Пройдем по узлам и создадим прямоугольники
for i in range(len(nodes_sorted_x) - 1):
    current_node = nodes_sorted_x[i]
    if nodes_sorted_x[i+1][0] == current_node[0]:
        next_y_node = nodes_sorted_x[i+1]
    else:
        continue
    # Найдем узлы следующие по оси y для текущих узлов
    next_x_nodes = [node for node in nodes_sorted_x if node[1] == current_node[1] and node[0] > current_node[0]]
    if next_x_nodes:
        next_x_node = min(next_x_nodes, key=lambda x: x[0])
        opposite_node = (next_x_node[0], next_y_node[1])
        # Построим прямоугольник по найденным узлам
        rectangle = ((current_node[0], current_node[1]), (opposite_node[0], opposite_node[1]))
        rectangles.append(rectangle)
        # cv2.rectangle(image, (current_node[0] + 2, height - current_node[1] + 2), (opposite_node[0] - 2, height - opposite_node[1] - 2), (255, 255, 0), 1)
    else:
        continue

rectangles = sorted(rectangles, key=lambda x: (- x[0][1], x[0][0]))

# # Отображение изображения
# cv2.imshow('Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[((8, 8), (177, 37)), ((8, 37), (177, 65)), ((8, 65), (177, 178)), ((177, 8), (424, 37)), ((177, 37), (424, 65)), ((177, 65), (424, 178)), ((424, 8), (670, 37)), ((424, 37), (670, 65)), ((424, 65), (670, 178)), ((670, 8), (839, 37)), ((670, 37), (839, 65)), ((670, 65), (839, 178))]


In [326]:
# Создание пустого массива для хранения данных таблицы
table_data = [['' for _ in range(num_columns)] for _ in range(num_rows)]

# Обход прямоугольников ячеек таблицы
for i, rectangle in enumerate(rectangles):
    # Извлечение координат углов прямоугольника
    x1, y1 = rectangle[0]
    x2, y2 = rectangle[1]

    # Обрезание изображения по границам ячейки
    cell_image = image[(height - y2):(height - y1), x1:x2]

    # Распознавание текста с помощью Tesseract
    custom_config = r'--oem 1 --psm 1 -l rus'
    text = pytesseract.image_to_string(cell_image, config=custom_config).replace('\n', ' ')

    # Определение положения ячейки в таблице
    row = i // num_columns
    column = i % num_columns

    # Добавление данных ячейки
    table_data[row][column] = text

# Вывод полученной таблицы
df = pd.DataFrame(table_data)
# Сохранение DataFrame в формате CSV
df.to_csv('table.csv', index=False, header=False)

Для выделения таблицы из корпуса текста попробуем использовать cv2 с такими настройками

In [332]:
import cv2
import numpy as np

# Загрузка изображения
image = cv2.imread('in_context.jpg')

# Преобразование изображения в оттенки серого
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Применение порогового преобразования для бинаризации изображения
ret, threshold_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Применение морфологического преобразования для удаления шума и замыкания линий
kernel = np.ones((5, 5), np.uint8)
morph_image = cv2.morphologyEx(threshold_image, cv2.MORPH_CLOSE, kernel)

# Нахождение контуров на изображении
contours, hierarchy = cv2.findContours(morph_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Создание списка координат ограничивающих рамок ячеек
cell_boxes = []

# Фильтрация контуров и определение ограничивающих рамок ячеек
for contour in contours:
    # Определение площади контура
    area = cv2.contourArea(contour)

    # Фильтрация контуров по площади
    if area > 3000:
        # Определение ограничивающей рамки контура
        x, y, w, h = cv2.boundingRect(contour)

        # Добавление ограничивающей рамки в список
        cell_boxes.append((x, y, x + w, y + h))

# Вывод координат ограничивающих рамок ячеек
for box in cell_boxes:
    x1, y1, x2, y2 = box
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Отображение изображения с выделенными ячейками
cv2.imshow('Image with Cells', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Сгенерируем датасет для оценки качества выполнения задачи. Возможно дообучим pytorch nn. Надо доработать генератор с учетом специфики конкретных таблиц ГОСТа

In [1]:
!pip install tabula - py

     ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.0 MB 1.3 MB/s eta 0:00:10
     ---------------------------------------- 0.1/12.0 MB 1.3 MB/s eta 0:00:10
      --------------------------------------- 0.2/12.0 MB 1.8 MB/s eta 0:00:07
      --------------------------------------- 0.3/12.0 MB 1.7 MB/s eta 0:00:07
     - -------------------------------------- 0.4/12.0 MB 1.9 MB/s eta 0:00:07
     - -------------------------------------- 0.5/12.0 MB 1.7 MB/s eta 0:00:07
     - -------------------------------------- 0.6/12.0 MB 1.7 MB/s eta 0:00:07
     -- ------------------------------------- 0.7/12.0 MB 1.9 MB/s eta 0:00:07
     --- ------------------------------------ 0.9/12.0 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.1/12.0 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.2/12.0 MB 2.2 MB/s eta 0:00:05
     ---- ----------------------------------- 1.3/12.0 MB 2

In [3]:
!pip install fpdf

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40713 sha256=3d030d429e9eb0115ed48106e615e2162d4ee5f59fce765845635860f679b694
  Stored in directory: c:\users\dsash\appdata\local\pip\cache\wheels\44\35\8b\86ce00cec7e4d13c5f189680ae0fa82f919bedc066c2cddae9
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.


In [18]:
import tabula
import pandas as pd
import random
import string
from fpdf import FPDF

# Функция для генерации случайного текста или цифр
def generate_random_data(length):
    data_type = random.choice(["letters", "digits"])
    if data_type == "digits":
        return ''.join(random.choice(string.digits) for _ in range(length))
    else:
        return ''.join(random.choice(string.ascii_letters) for _ in range(length))

# Функция для генерации случайной таблицы
def generate_random_table(num_rows, num_cols):
    table_data = []
    for _ in range(num_rows):
        row = [generate_random_data(random.randint(1, 5)) for _ in range(num_cols)]
        table_data.append(row)
    return table_data

# Функция для создания датасета
def generate_dataset(num_samples, rows, cols):
    # Создание директории "dataset", если она не существует
    directory = "dataset"
    if os.path.exists(directory):
        for file_name in os.listdir(directory):
            file_path = os.path.join(directory, file_name)
            os.remove(file_path)
    else:
        os.makedirs(directory)

    for i in range(num_samples):
        # Генерация случайной таблицы
        table_data = generate_random_table(rows, cols)
        df = pd.DataFrame(table_data)

        # Создание PDF файла с таблицей
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        for row in table_data:
            for item in row:
                pdf.cell(30, 10, str(item), border=1)
            pdf.ln()

        pdf_file = f"dataset/sample_table_{i}.pdf"
        pdf.output(pdf_file)

        # Создание CSV файла с меткой
        csv_file = f"dataset/sample_table"
        df.to_csv(f"{csv_file}_{i}.csv", index=False)

# Параметры генерации датасета
num_samples = 10  # Количество образцов
rows = 5  # Количество строк в таблице
cols = 5  # Количество столбцов в таблице

# Создание датасета
generate_dataset(num_samples, rows, cols)

Добавить в датасете таблицы разных размеров, шумы и искажения перспективы на изображениях